In [1]:
import requests
import pandas as pd
from requests.auth import HTTPBasicAuth
import os

In [2]:
# Read credentials from environment variables
username = os.getenv("OPENSKY_USERNAME")
password = os.getenv("OPENSKY_PASSWORD")

api_url = "https://opensky-network.org/api/states/all"

# Fetch data from OpenSky API
response = requests.get(api_url, auth=HTTPBasicAuth(username, password), timeout=30)
data = response.json()

print(data.keys()) # Available data keys

dict_keys(['timestamp', 'status', 'error', 'message', 'path'])


In [3]:
print(response.status_code)

401


In [4]:
# Fetch data from the API
try:
    response = requests.get(api_url, timeout=20)
    response.raise_for_status()
    flight_data = response.json()
except requests.exceptions.RequestException as error:
    print("Error accessing API:", error)
    flight_data = {"states": []}

# Define expected DataFrame columns
columns = [
    "icao24", "callsign", "origin_country", "time_position", "last_contact",
    "longitude", "latitude", "baro_altitude", "on_ground", "velocity",
    "true_track", "vertical_rate", "sensors", "geo_altitude",
    "squawk", "spi", "position_source"]

# Create DataFrame
df = pd.DataFrame(flight_data.get("states", []), columns=columns)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9106 entries, 0 to 9105
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   icao24           9106 non-null   object 
 1   callsign         9106 non-null   object 
 2   origin_country   9106 non-null   object 
 3   time_position    9031 non-null   float64
 4   last_contact     9106 non-null   int64  
 5   longitude        9031 non-null   float64
 6   latitude         9031 non-null   float64
 7   baro_altitude    8256 non-null   float64
 8   on_ground        9106 non-null   bool   
 9   velocity         9104 non-null   float64
 10  true_track       9106 non-null   float64
 11  vertical_rate    8284 non-null   float64
 12  sensors          0 non-null      object 
 13  geo_altitude     8218 non-null   float64
 14  squawk           3921 non-null   object 
 15  spi              9106 non-null   bool   
 16  position_source  9106 non-null   int64  
dtypes: bool(2), fl

In [6]:
import numpy as np

# Drop empty columns e.g.'sensors'
df = df.dropna(axis=1, how="all")

# Fill missing numerical values with their median
df = df.fillna(df.select_dtypes(include='number').median())

# Fill missing categorical values with 'NULL'
df[df.select_dtypes(include="object").columns] = (df.select_dtypes(include="object").fillna("NULL"))

In [7]:
# Verify no missing values
if df.isnull().sum().sum() != 0:
    raise ValueError("DataFrame contains missing values")

In [8]:
from sqlalchemy import create_engine

# PostgreSQL credentials
username      = "postgres" 
password      = "postgre2026"
host          = "localhost"
port          = "5432"
database      = "OpenSkyNetwork"

engine = create_engine(f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}")

# Load data into PostgreSQL
table_name = "data"
df.to_sql(table_name, engine, if_exists="replace", index=False)

print(f"Data succesfully loaded into table 'data' in database '{database}'.")

Data succesfully loaded into table 'data' in database 'OpenSkyNetwork'.


## Author

<a href="https://www.linkedin.com/in/andrew-kalumba-harris/">Andrew Kalumba</a><br>
<a href =""> </a>


| Date (YYYY-MM-DD) | Prepared By     | 
| ----------------- | --------------  | 
| 2026-01-31        | Author          | 


## <h3 align="center"> © Data Science 2026. <h3/>